### 9.3 BERT ile Soru Cevaplama (Question Answering)

In [ ]:
# !pip install numpy==1.24.4 --force-reinstall

from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import warnings
warnings.filterwarnings("ignore")

model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

def predict_answer(context, question):
    encoding = tokenizer.encode_plus(
        question, context,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    with torch.no_grad():
        start_scores, end_scores = model(input_ids, attention_mask=attention_mask, return_dict=False)

    start_index = torch.argmax(start_scores, dim=1).item()
    end_index = torch.argmax(end_scores, dim=1).item()

    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start_index: end_index + 1])
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer

# Test 1
question = "What is the capital of France"
context = "France, officially the French Republic, is a country whose capital is Paris"
answer = predict_answer(context, question)
print(f"Answer: {answer}")

# Test 2
question = "What is Machine Learning?"
context = """ Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance 
on a specific task. Machine learning algorithms build a mathematical model of sample data, known as 'training data', in order to make predictions or 
decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection 
of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning 
is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, 
theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory 
data analysis through unsupervised learning. In its application across business problems, machine learning is also referred to as predictive analytics. """
answer = predict_answer(context, question)
print(f"Answer: {answer}")


c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: paris
Answer: the scientific study of algorithms and statistical models


#### 9.4 GPT ile Soru Cevaplama (Question Answering)

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

import warnings
warnings.filterwarnings("ignore")

model_name = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_answer(context, question):
    
    input_text = f"Question: {question}, Context: {context}. Please answer the question according to context"
    
    # tokenize
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    
    # modeli calistir
    with torch.no_grad():
        outputs = model.generate(inputs, max_length = 300)
        
    # uretilen yaniti decode edelim
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True) # merhaba<EOS><PAD>
    
    # tanitlari ayiklayalim
    answer = answer.split("Answer:")[-1].strip()
    
    return answer

question = "What is the capital of France"
context = "France, officially the French Republic, is a country whose capital is Paris"

answer = generate_answer(context, question)
print(f"Answer: {answer}")

question = '''What is Machine Learning?'''
context = ''' Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance 
                on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or 
                decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection 
                of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning 
                is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, 
                theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory 
                data analysis through unsupervised learning.In its application across business problems, machine learning is also referred to as predictive analytics. '''

answer = generate_answer(context, question)
print(f"Answer: {answer}")  

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: The capital of France is Paris.

Question: What is the capital of France, Context: France, officially the French Republic,
Answer: Question: What is Machine Learning?, Context:  Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance 
                on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or 
                decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection 
                of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning 
                is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimiz

### 9.5 Bilgi Getirimi ( Information Retrieval )

![ScreenS/9.5_info.PNG](ScreenS/9.5_info.PNG)

In [1]:
from transformers import BertTokenizer, BertModel

import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

# tokenizer and model create 
model_name = "bert-base-uncased" # kucuk boyutlu bert modeli 
tokenizer = BertTokenizer.from_pretrained(model_name) # tokenizer yukle
model = BertModel.from_pretrained(model_name) # onceden egitilmis bert modeli 

# veri olustur: karsılastırılcak belgeleri ve sorgu cumlemizi olustur 
documents = [
    "Machine learning is a field of artificial intelligence",
    "Natural language processing involves understanding human language",
    "Artificial intelligence encomppases machine learning and natural language processing (nlp)",
    "Deep learning is a subset of machine learning",
    "Data science combines statistics, adta analysis and machine learning",
    "I go to shop"
]

query = "What is deep learning?"

# bert ile bilgi getirimi

def get_embedding(text):
    # tokenization
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # modeli calistir
    outputs = model(**inputs)

    # son gizli katmani alalim
    last_hidden_state = outputs.last_hidden_state
    
    # metni temsili olustur
    embedding = last_hidden_state.mean(dim=1)
    
    # vektoru numpy olarak return et
    return embedding.detach().numpy()

# belgeler ve sorgu icin embedding vektorlerini al
doc_embeddings = np.vstack([get_embedding(doc) for doc in documents])
query_embedding = get_embedding(query)

# kosinus benzerligi ile belgeler arasinda benzerligi hesaplayalim
similarities = cosine_similarity(query_embedding, doc_embeddings)

# her belgenin benzerlik skoru
for i, score in enumerate(similarities[0]):
    print(f"Document: {documents[i]} \n{score}")


Document: Machine learning is a field of artificial intelligence 
0.634821891784668
Document: Natural language processing involves understanding human language 
0.626939058303833
Document: Artificial intelligence encomppases machine learning and natural language processing (nlp) 
0.5046247243881226
Document: Deep learning is a subset of machine learning 
0.6263622641563416
Document: Data science combines statistics, adta analysis and machine learning 
0.6136887073516846
Document: I go to shop 
0.5354946255683899


In [2]:
most_similar_index = similarities.argmax()

print(f"Most similar document: {documents[most_similar_index]}")

Most similar document: Machine learning is a field of artificial intelligence


### 9.7 Derin Ogrenme ile Öneri sistemleri ( Recommendation system )

In [ ]:
"""
    Problem tanimi: Oneri sistemi
    user (kullanici) - item (urunler) - rating (puanlar)
"""

import numpy as np 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input,Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

# veri setini olustur
user_ids = np.array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4])
item_ids = np.array([0, 1, 2, 3, 4, 1, 2, 3, 4, 5])
ratings  = np.array([5, 4, 3, 2, 1, 4, 5, 3, 2, 1])

# train test split
user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test = train_test_split(user_ids, item_ids, 
                                                                                                             ratings, test_size = 0.2, 
                                                                                                             random_state = 42)

# creaate neural network 
def create_model(num_users, num_items, embedding_dim):

    # input katmani
    user_input = Input(shape=(1,), name = "user")
    item_input = Input(shape=(1,), name = "item")

    # embedding katmani 
    user_embedding = Embedding(input_dim = num_users, output_dim = embedding_dim, name = "user_embedding")(user_input)
    item_embedding = Embedding(input_dim = num_items, output_dim = embedding_dim, name = "item_embedding")(item_input)

    # vektorleri duzlestir 
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

    dot_product = Dot(axes = 1)([user_vecs, item_vecs])
    output = Dense(1)(dot_product)

    model = Model(inputs = [user_input, item_input], outputs = output)
    model.compile(optimizer=Adam(learning_rate = 0.01), loss = "mean_squared_error")
    return model
# train and test 

num_users = 5
num_items = 6
embedding_dim = 8
model = create_model(num_users, num_items, embedding_dim)
model.fit([user_ids_train, item_ids_train], ratings_train, epochs = 100, verbose = 1, validation_split = 0.1)

loss = model.evaluate([user_ids_test, item_ids_test], ratings_test)
print(f"test: {loss}")

# recommendation test
user_id = np.array([0])
item_id = np.array([0])
prediction = model.predict([user_id, item_id])

print(f"Predicted rating for user: {user_id[0]}, item: {item_id[0]}, {prediction[0][0]:.2f}")


Epoch 1/100

1/1 [==============================] - 1s 1s/step - loss: 9.2845 - val_loss: 24.9078
Epoch 2/100
1/1 [==============================] - 0s 43ms/step - loss: 9.2183 - val_loss: 24.8087
Epoch 3/100
1/1 [==============================] - 0s 45ms/step - loss: 9.1513 - val_loss: 24.7098
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 9.0809 - val_loss: 24.6108
Epoch 5/100
1/1 [==============================] - 0s 46ms/step - loss: 9.0059 - val_loss: 24.5119
Epoch 6/100
1/1 [==============================] - 0s 47ms/step - loss: 8.9255 - val_loss: 24.4129
Epoch 7/100
1/1 [==============================] - 0s 45ms/step - loss: 8.8392 - val_loss: 24.3142
Epoch 8/100
1/1 [==============================] - 0s 46ms/step - loss: 8.7466 - val_loss: 24.2157
Epoch 9/100
1/1 [==============================] - 0s 43ms/step - loss: 8.6472 - val_loss: 24.1176
Epoch 10/100
1/1 [==============================] - 0s 45ms/step - loss: 8.5406 - val_loss: 24.0200
Epoch 11/

### 9.9 Makine ogrenmesi ile Oneri sistemi ( Recommendation system )

In [ ]:
# import libraries
from surprise import Dataset, KNNBasic, accuracy
from surprise.model_selection import train_test_split

# import veri seti
data = Dataset.load_builtin("ml-100k") # (kullanici id, film id, puan)

# train test split
trainset, testset = train_test_split(data, test_size = 0.2) 

# ML model create: KNN
model_options = {
    "name": "cosine",
    "user_based":True # kullanicilar arasi benzerlik
    }

# train
model = KNNBasic(sim_options=model_options)
model.fit(trainset)

# test
prediction = model.test(testset)
accuracy.rmse(prediction)

# recommendation system 
def get_top_n(predictions, n = 10):
    
    top_n = {}
    
    for uid, iid, true_r, est, _ in predictions:
        if not top_n.get(uid):
            top_n[uid] = []
        top_n[uid].append((iid, est))
        
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

n = 5
top_n = get_top_n(prediction, n)

user_id = "2"
print(f"top {n} recommendation for user {user_id}")
for item_id, rating in top_n[user_id]:
    print(f"item id: {item_id}, score: {rating}")
    
    
"""
RMSE: 0.98  # (bu sayı değişebilir, modelin tahmin doğruluğu)

top 5 recommendation for user 2
item id: 169, score: 5.0
item id: 174, score: 5.0
item id: 98, score: 5.0
item id: 96, score: 4.99
item id: 318, score: 4.99

"""

### Makine Çevirisi ( Machine Translation )

![ScreenS/9.10_mt.PNG](ScreenS/9.10_mt.PNG)

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-fr" # eng to fr --- "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

text = "Hello, what is your name"

# encode edelim, sonrasinda modele input olarak verelim
translated_text = model.generate(**tokenizer(text, return_tensors="pt", padding = True))

# translated text metne donusturulur
translated_text = tokenizer.decode(translated_text[0], skip_special_tokens=True)
print(f"Translated_text: {translated_text}")

c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mfurk\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-fr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\m

Translated_text: Bonjour, quel est votre nom


### 9.12 Metin Özetleme ( Text Summarization )

![ScreenS/9.12_TS.PNG](ScreenS/9.12_TS.PNG)

In [2]:
from transformers import pipeline 

# ozetleme pipeline yukle
summarizer = pipeline("summarization")

text = """
Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use 
to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical 
model of sample data, known as "training data", in order to make predictions or decisions without being explicitly 
programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, 
detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific 
instructions for performing the task. Machine learning is closely related to computational statistics, which focuses 
on making predictions using computers. The study of mathematical optimization delivers methods, theory and application 
domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory 
data analysis through unsupervised learning. In its application across business problems, machine learning is also referred 
to as predictive analytics.
"""

# metni ozetleme
summary = summarizer(
    text, 
    max_length = 20,
    min_length = 5,
    do_sample = True
    )

print(summary[0]["summary_text"])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\mfurk\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mfurk\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activa

 Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems


### 9.14 ChatBotGeliştirme ve API Kullanımı 

In [ ]:
import openai

openai.api_key = "api-key"

def chat_with_gpt(prompt, history_list):
    
    responce = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{"role":"user", "content": f"Bu bizim mesajımız: {prompt}. Konuşma geçmişi: {history_list}"}]
        )
    
    return responce.choices[0].message.content.strip()

if __name__ == "__main__":
    
    history_list = []
    
    while True:
        
        user_input = input("Kullanıcı tarafından girilen mesaj:" )
        
        if user_input.lower() in ["exit", "q"]:
            print("Konuşma tamamlandi")
            break
        history_list.append(user_input)
        responce = chat_with_gpt(user_input, history_list)
        print(f"Chatbot: {responce}")
